# Example: CitiBike data

Adapted from Kelsey Jordahl
https://gist.github.com/kjordahl/5957573

In [2]:
import requests  
import json
from pyproj import Proj
from shapely.geometry import Point
import geopandas as gp

NYC borough boundaries downloaded from [Bytes of the Big Apple](http://www.nyc.gov/html/dcp/download/bytes/nybb_13a.zip)

In [3]:
boros = gp.GeoDataFrame.from_file('nybb_15b/nybb.shp')

Load real time bike station data from [CitiBike](http://citibikenyc.com) json API:

In [6]:
# get the bike data and convert it to a ditcionary
endpoint_url = 'http://citibikenyc.com/stations/json'
response = requests.get(endpoint_url)
data = json.loads(response.text)
data

{'executionTime': '2018-02-21 05:25:15 PM',
 'stationBeanList': [{'altitude': '',
   'availableBikes': 3,
   'availableDocks': 36,
   'city': '',
   'id': 72,
   'landMark': '',
   'lastCommunicationTime': '2018-02-21 05:23:40 PM',
   'latitude': 40.76727216,
   'location': '',
   'longitude': -73.99392888,
   'postalCode': '',
   'stAddress1': 'W 52 St & 11 Ave',
   'stAddress2': '',
   'stationName': 'W 52 St & 11 Ave',
   'statusKey': 1,
   'statusValue': 'In Service',
   'testStation': False,
   'totalDocks': 39},
  {'altitude': '',
   'availableBikes': 25,
   'availableDocks': 7,
   'city': '',
   'id': 79,
   'landMark': '',
   'lastCommunicationTime': '2018-02-21 05:23:54 PM',
   'latitude': 40.71911552,
   'location': '',
   'longitude': -74.00666661,
   'postalCode': '',
   'stAddress1': 'Franklin St & W Broadway',
   'stAddress2': '',
   'stationName': 'Franklin St & W Broadway',
   'statusKey': 1,
   'statusValue': 'In Service',
   'testStation': False,
   'totalDocks': 33},

In [5]:
# convert the relevant part to a geodataframe
df = gp.GeoDataFrame(data['stationBeanList'])
df.head()

,altitude,availableBikes,availableDocks,city,id,landMark,lastCommunicationTime,latitude,location,longitude,postalCode,stAddress1,stAddress2,stationName,statusKey,statusValue,testStation,totalDocks
0,,4,35,,72,,2018-02-21 05:20:34 PM,40.767272,,-73.993929,,W 52 St & 11 Ave,,W 52 St & 11 Ave,1,In Service,False,39
1,,25,7,,79,,2018-02-21 05:19:55 PM,40.719116,,-74.006667,,Franklin St & W Broadway,,Franklin St & W Broadway,1,In Service,False,33
2,,14,12,,82,,2018-02-21 05:21:54 PM,40.711174,,-74.000165,,St James Pl & Pearl St,,St James Pl & Pearl St,1,In Service,False,27
3,,30,30,,83,,2018-02-21 05:20:38 PM,40.683826,,-73.976323,,Atlantic Ave & Fort Greene Pl,,Atlantic Ave & Fort Greene Pl,1,In Service,False,62
4,,7,11,,119,,2018-02-21 05:20:18 PM,40.696089,,-73.978034,,Park Ave & St Edwards St,,Park Ave & St Edwards St,1,In Service,False,19


In [7]:
# there is one row for each bike station.  How many stations are there? 
len(df)

813

In [17]:
df.to_crs?

In [10]:
# in the file above, there are lon-lats, but no geometry field
# we need to set that up

s = gp.GeoSeries([Point(x, y) for x, y in zip(df['longitude'], df['latitude'])])
df['geometry'] = s
df.crs = {'init': 'epsg:4326', 'no_defs': True}
df.geometry.total_bounds

array([-74.0969366 ,  40.65539977, -73.9077436 ,  40.81439444])

In [16]:
# make sure they are on the same CRS.  
# checking the bounds is a nice way of seeing this
df.to_crs(boros.crs, inplace=True)
df.geometry.total_bounds

array([  957370.14732176,   178056.37414829,  1009803.53178432,
         235986.42166716])

In [18]:
# the geometry objects can do lots of cool stuff.  For example: 

manhattan = boros.geometry[3]
in_mn = df.geometry.within(manhattan)
print(sum(in_mn), 'stations in Manhattan')

414 stations in Manhattan


In [27]:
df.geometry.within?


# Your turn

You can read about the range of operations available in geopandas here: 
    
http://geopandas.org/index.html

Your assignment is to: 

1. Calculate how many stations are in each borough
2. Calculate how many bikes are currently available in each borough
3. Read about and try at least two new spatial or geometric operations (beyond what I've covered here). 

In [19]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gp
from shapely.geometry import Point
import pandas as pd

In [20]:
boros = gp.read_file('nybb_15b/nybb.shp')
boros

,BoroCode,BoroName,Shape_Leng,Shape_Area,geometry
0,5,Staten Island,330466.075042,1.623827e+09,(POLYGON ((970217.0223999023 145643.3322143555...
1,3,Brooklyn,741185.900596,1.937597e+09,(POLYGON ((1021176.479003906 151374.7969970703...
2,4,Queens,897040.298576,3.045168e+09,(POLYGON ((1029606.076599121 156073.8142089844...
3,1,Manhattan,358408.460709,6.364467e+08,(POLYGON ((981219.0557861328 188655.3157958984...
4,2,Bronx,464400.198868,1.186973e+09,(POLYGON ((1012821.805786133 229228.2645874023...


In [21]:
# set the index to be the name of the borough, and sort in order of the borough code
boros.reset_index(inplace=True)
boros.set_index('BoroName', inplace=True)
boros.sort_values(by='BoroCode', inplace=True)
boros

,index,BoroCode,Shape_Leng,Shape_Area,geometry
BoroName,,,,,
Manhattan,3,1,358408.460709,6.364467e+08,(POLYGON ((981219.0557861328 188655.3157958984...
Bronx,4,2,464400.198868,1.186973e+09,(POLYGON ((1012821.805786133 229228.2645874023...
Brooklyn,1,3,741185.900596,1.937597e+09,(POLYGON ((1021176.479003906 151374.7969970703...
Queens,2,4,897040.298576,3.045168e+09,(POLYGON ((1029606.076599121 156073.8142089844...
Staten Island,0,5,330466.075042,1.623827e+09,(POLYGON ((970217.0223999023 145643.3322143555...


In [22]:
station_mn = df.geometry.within(boros.geometry[0])
station_brnx = df.geometry.within(boros.geometry[1])
station_queens = df.geometry.within(boros.geometry[3])
station_sttn = df.geometry.within(boros.geometry[4])
station_brkln = df.geometry.within(boros.geometry[2])

print(sum(station_sttn),"stations in staten island",sum(station_mn),"in manhattan",sum(station_brnx),"in bronx",sum(station_brkln),"in brooklyn")

0 stations in staten island 414 in manhattan 0 in bronx 266 in brooklyn


In [42]:
total=0
#count=0
for row in range(0,len(df)):
    if df['geometry'][row].within(boros.geometry[2])==True:
        total=total+df['availableBikes'][row]
        #count=count+1
print ("total available bikes in brooklyn",total)

total available bikes in brooklyn 1848
